In [1]:
import json
import tarfile
import numpy as np
import matplotlib.pyplot as plt
import random
import textwrap
from pathlib import Path
import pandas as pd
from PIL import Image
from io import BytesIO
from IPython.display import HTML
import base64
pd.set_option('display.max_colwidth', None)

In [21]:
def image_base64(img):
    with BytesIO() as buffer:
        img.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()
    
def image_formatter(img):
    return f'<img src="data:image/jpeg;base64,{image_base64(img)}" width="500">'

def show_sample_examples(wds_path, num_samples=1, different_tars=False):
    IMAGE_EXTENSIONS = {"JPG", "jpg", "png", "jpeg", "bmp", "tif", "tiff"}
    
    all_tarpaths = list(Path(wds_path).glob("**/*.tar"))
    
    if different_tars:
        tarfps = random.sample(all_tarpaths, min(num_samples, len(all_tarpaths)))
        tarfps_samples = [(tarfp, 1) for tarfp in tarfps]
    else:
    
        tarfps_samples = [(random.choice(all_tarpaths), num_samples)]
        
    print(tarfps_samples)
    
    data = []
    
    for tarfp, num_samples_for_tar in tarfps_samples:
        with tarfile.open(tarfp, 'r') as tar:
            sampled_members = random.sample([member for member in tar.getmembers() if member.name.endswith(".json")],
                                            min(len(tar.getnames())//2, num_samples_for_tar))
            
            print("here:", tarfp)
            
            for member in sampled_members:
                print("member.name:", member.name)
                key, _ = member.name.split(".")

                
                key = key.split("/")[-1]
                print("key", key)
                
                example = json.load(tar.extractfile(f"{key}.json"))   
                
                img_fn = next(
                            (
                                f"{key}.{ext}"
                                for ext in IMAGE_EXTENSIONS
                                if f"{key}.{ext}" in tar.getnames()
                            ),
                            None,
                        )
        
                img = Image.open(tar.extractfile(img_fn)).convert("RGB")
                img.load()
                example["__tarfile__"] = tarfp
                example["__key__"] = key
                example["image"] = img
                data.append(example)
                
    cols = data[0].keys()
    data_dict = {col: [sample.get(col, None) for sample in data] for col in cols} 
    df = pd.DataFrame(data=data_dict)
    return df

# test_path1 = "/fsx_3/dataset01/pdfa-eng-wds-converted"
# test_path2 = "/fsx_3/bucket/qyh/landmark_all"
path = "/fsx_3/bucket/infoseek/infoseekQA"
df = show_sample_examples(path)
HTML(df.to_html(formatters={'image': image_formatter}, escape=False))

[(PosixPath('/fsx_3/bucket/infoseek/infoseekQA/00050/00050_batch_5.tar'), 1)]
here: /fsx_3/bucket/infoseek/infoseekQA/00050/00050_batch_5.tar
member.name: 00050/infoseek_train_00664941.json
key infoseek_train_00664941


KeyError: "filename 'infoseek_train_00664941.json' not found"

[(PosixPath('/fsx_3/bucket/infoseek/infoseekQA/00000/00000_batch_11.tar'), 1)]
here
key 00000/infoseek_test_00008520


AttributeError: 'NoneType' object has no attribute 'isreg'

In [ ]:
# captioning
sample = {
    "image": <img>,
    "text": "Bobbin lace linen. Flanders, early 18th Century. | Costumes and fashions of the centuries."
}

# instruction_tune
sample = {
    "image": <img>,
    "instruction": "Answer the question using a single word or phrase.",
    "question": "What formation of planes is this?",
    "response": "square"
}